In [ ]:
import pandas as pd
# On charge le fichier csv qui se trouve dans un sous-dossier 'data'
data = pd.read_csv("./data/GlobalTemperatures.csv")
# On charge les températures moyennes (terre et océan confondus) dans une série de données Pandas,
# à partir de la 1300ème ligne (il n'y a aucune donnée avant cela).
# On réalise aussi une interpolation pour remplir automatiquement certaines valeurs manquantes
temperature = data["LandAndOceanAverageTemperature"].interpolate(limit_direction="backward")[1300:]
dates = pd.to_datetime(data["dt"])[1300:]

In [ ]:
from statsmodels.tsa.stattools import adfuller

def make_stationary(time_series):
    """
    Makes a time series stationary while the p-value computed using the ADF test is higher than 0.05
    """
    test_results = adfuller(time_series)
    p_value = test_results[1]
    if p_value > 0.05: # i.e. if the data is not stationary
        # Differentiating until the p-value goes under 0.05
        diff_data = time_series.copy()
        for degree in range(1, 10):
            diff_data = diff_data.diff().dropna()
            if adfuller(diff_data)[1] <= 0.05:
                return diff_data, degree
        
        raise ValueError(f"Unable to stationarize data after diferentiating {degree} times")
    return time_series

_, diff_degree = make_stationary(temperature)
print(f"Degrees of differentiating: {diff_degree}")

In [ ]:
from pmdarima.model_selection import train_test_split

# On commence par séparer nos jeux d'entraînement et de test
training_samples_count = int(len(dates) * 0.9)
train, test = train_test_split(temperature, train_size=training_samples_count)
train_dates, future_dates = train_test_split(dates, train_size=training_samples_count)

In [ ]:
# Recherche des meilleurs paramètres pour notre ARIMA
# Comme nous avons déjà déterminé que notre degré de différenciation
# est de 1, nous pouvons le préciser à auto_arima pour gagner du temps
# d'exécution
from pmdarima import auto_arima
# On choisit une valeur de 12 pour la saisonnalité car nos données
# sont des moyennes mensuelles
best_model = auto_arima(train, d=1, seasonal=True, m=12, stepwise=False, n_jobs=-1, trace=True)
print(best_model)

In [ ]:
from pmdarima.arima import ARIMA

# On instancie notre modèle avec les hyperparamètres définis plus tôt
model = ARIMA(
    order=(1, 1, 1),
    seasonal_order=(2, 0, 1, 12)
               )
model.fit(train)

predictions_count = len(test)
# On réalise des prédictions à comparer avec les valeurs de test
forecast, confidence_intervals = model.predict(predictions_count, return_conf_int=True) 


In [ ]:
# Adding confidence intervals
ci_dict = {
    "low": confidence_intervals[:, 0],
    "high": confidence_intervals[:, 1]
}
confidence_intervals_df = pd.DataFrame(ci_dict)
confidence_intervals_df.index = future_dates

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = np.arange(len(train_dates) + len(future_dates))
trend = np.polyfit(x, pd.concat([train, forecast]), 1)


trendpoly = np.poly1d(trend)
plt.plot(dates, temperature, c='blue')
plt.plot(future_dates, forecast, c='green')
plt.plot(dates, trendpoly(x), c="red")
plt.fill_between(confidence_intervals_df.index, confidence_intervals_df['low'], confidence_intervals_df['high'], alpha=0.9, color='orange', label="Confidence intervals")
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


plt.figure(figsize=(15,8))



# Computing yearly average in order to increase readibility
temperature.index = dates
yearly_temperature = temperature.rolling(window=12, step=12, center=False).mean().dropna()

train.index = train_dates
yearly_train = train.rolling(window=12, step=12, center=False).mean().dropna()

forecast.index = future_dates
yearly_forecast = forecast.rolling(window=12, step=12, center=False).mean().dropna()

train_and_forecast = pd.concat([yearly_train, yearly_forecast])
x = np.arange(len(train_and_forecast))
trend = np.polyfit(x, train_and_forecast, 1)
trendpoly = np.poly1d(trend)

yearly_train_dates = pd.Series(yearly_train.index)
yearly_forecast_dates = pd.Series(yearly_forecast.index)
all_dates = pd.concat([yearly_train_dates, yearly_forecast_dates])

yearly_confidence_intervals_df = confidence_intervals_df.rolling(window=12, step=12, center=False).mean().dropna()

plt.plot(yearly_temperature, c='blue', label='Températures annuelles réelles')
plt.plot(yearly_forecast_dates, yearly_forecast.values, c='green', label='Températures annuelles prédites')
#plt.plot(train_and_forecast.index, trendpoly(x), c="red", label='Forecast trend')
plt.fill_between(yearly_confidence_intervals_df.index, yearly_confidence_intervals_df['low'], yearly_confidence_intervals_df['high'], alpha=0.9, color='orange', label="Intervalles de confiance")
plt.legend(loc='upper left')
plt.gcf().autofmt_xdate()


In [ ]:
# Calcul de la RMSE
import math
from sklearn.metrics import mean_squared_error

def rmse(true_values, predicted_values):
    """
    Computes the Root Mean-Squared Error of the predicted values with regards to the actual values.
    """
    return math.sqrt(mean_squared_error(true_values, predicted_values))

# Exemple avec nos prédictions :
print(f"RMSE : {rmse(test, forecast)}")


## Importations

In [ ]:
import pandas as pd
from pmdarima import auto_arima
from pmdarima.arima import ARIMA
from pmdarima.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import SGDRegressor
import numpy as np
import matplotlib.pyplot as plt

SPLIT_DATASET = True # Use this if you want to perform tests on parts of the time series
DATASET_SPLIT_RATIO = 0.85 # If SPLIT_DATASET, is set to True, defines how much of the data should be used for training
FORECAST_LENGTH = 120 # Number of forecast samples to generate if SPLIT_DATASET is set to False


In [ ]:
def make_stationary(time_series):
    """
    Make a time series stationary if the p-value computed using the ADF test is higher than 0.05
    """
    test_results = adfuller(time_series)
    p_value = test_results[1]
    if p_value > 0.05: # i.e. if the data is not stationary
        # Differentiating until the p-value goes under 0.05
        diff_data = time_series.copy()
        for degree in range(1, 10):
            diff_data = diff_data.diff().dropna()
            if adfuller(diff_data)[1] <= 0.05:
                return diff_data, degree
        
        raise ValueError(f"Unable to stationarize data after diferentiating {degree} times")
    return time_series



def decompose_data(time_series):
    """
    Decompose our data following an additive model
    """
    additive_decomposition = seasonal_decompose(time_series.values, model='additive', period=12)
    return {
        "trend": additive_decomposition.trend,
        "seasonal": additive_decomposition.seasonal,
        "residual": additive_decomposition.resid
    }
        

## Chargement des données

In [ ]:
data = pd.read_csv("./data/GlobalTemperatures.csv")
print(data.head())
# Getting rid of empty values (the first 1300 or so are missing)
temp = data["LandAndOceanAverageTemperature"].interpolate(limit_direction="backward")[1300:]
dates = pd.to_datetime(data["dt"])[1300:]

if SPLIT_DATASET:
    TRAINING_SAMPLES_COUNT = int(len(dates) * DATASET_SPLIT_RATIO)
    train, test = train_test_split(temp, train_size=TRAINING_SAMPLES_COUNT)
    train_dates, future_dates = train_test_split(dates, train_size=TRAINING_SAMPLES_COUNT)


else:
    train = temp
    train_dates = dates

# Making our series stationary
train_components = decompose_data(train)
detrended_train = train.values - train_components["trend"]
trend_train = pd.Series(train_components["trend"])
detrended_train = pd.Series(detrended_train).reset_index(drop=True)



## Recherche des meilleurs paramètres et entraînement

Paramètres du meilleur modèle :
Best model:  ARIMA(1,1,1)(2,0,2)[12] intercept

In [ ]:
trend_train = trend_train.dropna()
print(np.array(trend_train.index).reshape(-1, 1))
print(np.array(trend_train.values))

#model = auto_arima(train, seasonal=True, m=12, trace=True)
#print(str(model.summary()))
model = ARIMA(order=(1, 1, 1), seasonal_order=(2, 0, 2, 12))
model.fit(detrended_train)

# ENTRAÎNER UN MODÈLE DE RÉGRESSION SUR LA TENDANCE ICI :
trend_model = SGDRegressor()
trend_model.fit(np.array(trend_train.index).reshape(-1, 1), trend_train.interpolate().values)


## Prédiction

In [ ]:
if SPLIT_DATASET:
    detrended_forecasts = model.predict(len(test))  # predict N steps into the future
    future_index = np.arange(trend_train.index[-1]+1, trend_train.index[-1]+1 + len(test)).reshape(-1, 1)
    trend_forecasts = trend_model.predict(future_index)

    forecasts = detrended_forecasts + trend_forecasts
    print(forecasts)
else:
    forecasts = model.predict(FORECAST_LENGTH)  # predict 10 years into the future

forecasts = pd.Series(forecasts)

## Visualisation

In [ ]:
# Visualize the forecasts (blue=train, green=forecasts)
plt.figure(figsize=(15,8))


if SPLIT_DATASET:
    x = np.arange(len(train) + len(forecasts))
    trend = np.polyfit(x, pd.concat([train, forecasts]), 2)

else:
    # Generating extra dates for the forecast
    start_date = dates.iloc[-1] + pd.DateOffset(months=1)
    future_dates = pd.date_range(start_date, periods=FORECAST_LENGTH, freq="M").to_series()
    all_dates = pd.concat([dates, future_dates])
    x = np.arange(len(train) + len(forecasts))
    trend = np.polyfit(x, pd.concat([train, forecasts]), 2)




trendpoly = np.poly1d(trend)
plt.plot(train_dates, train, c='blue')
plt.plot(future_dates, forecasts, c='green')
plt.plot(all_dates, trendpoly(x), c="red")
plt.gcf().autofmt_xdate()
plt.show()

In [ ]:
# Reintegrating trends + comparing forecasts to actual values

if SPLIT_DATASET:

    trend_data = pd.Series(components["trend"])
    all_temp = pd.concat([train, forecasts]).reset_index().drop(columns="index")[0]

    full_data = all_temp + trend_data


detrended_data = {
    "date": all_dates,
    "detrended_temp": pd.concat([train, forecasts])
}

detrended_df = pd.DataFrame(detrended_data)
print(detrended_df.shape)
plt.plot(detrended_df)


In [ ]:
# Visualisation en faisant des moyennes de température annuelles, pour plus de visibilité
plt.figure(figsize=(15,8))

train.index = train_dates
yearly_train = train.rolling(window=12, step=12, center=True).mean().dropna()

forecasts.index = future_dates
yearly_forecasts = forecasts.rolling(window=12, step=12, center=True).mean().dropna()

all_data = pd.concat([yearly_train, yearly_forecasts])
x = np.arange(len(all_data))

plt.plot(all_data)
plt.gcf().autofmt_xdate()

